# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

In [2]:
for k, v in obj.items():
    print(f"{k:10} = '{v}'")

home_page  = 'https://github.com/pypa/sampleproject'
keywords   = 'sample setuptools development'
license    = 'MIT'


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [3]:
import pandas as pd

In [4]:
import re

In [5]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

In [6]:
group_numbers = obj.str.extract(r"(\d{2}-\d)") # Найти две цифры подряд, затем дефис и еще одну цифру

print(group_numbers)

      0
0  19-1
1  20-4
2  20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [7]:
sentence = "Написать регулярное выражение, которое позволит найти номера групп студентов"
words = re.findall(r'\w+', sentence) # Найти все последовательности символов, которые являются буквами или цифрами
for word in words:
    print(word)

Написать
регулярное
выражение
которое
позволит
найти
номера
групп
студентов


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [19]:
# Загрузка набора данных
recipes = pd.read_csv('recipes_sample.csv')

# Выбор случайной выборки из 5 рецептов
recipes_5 = recipes.sample(5)

# Определение ширины столбцов для вывода таблицы
width = [
    max([len(str(i)) if len(str(i)) > 2 else 2 for i in recipes_5['id']]), 
    max([len(str(i)) if len(str(i)) > 7 else 7 for i in recipes_5['minutes']])
]

# Заголовки таблицы с выравниванием по центру и шириной, определенной выше
print(f'|{" id ":^{width[0] + 2}}|{"minutes":^{width[1] + 2}}|')
# Разделительная строка таблицы с выравниванием по центру и шириной, определенной выше
print(f'|{"-"*(width[0] + 2)}|{"-"*(width[1] + 2)}|')
# Вывод содержимого таблицы с выравниванием по центру и шириной, определенной выше
for i, j in zip(recipes_5['id'], recipes_5['minutes']):
    print(f'|{i:^{width[0] + 2}}|{j:^{width[1] + 2}}|')


|   id   | minutes |
|--------|---------|
| 61275  |   135   |
| 124044 |   120   |
| 337644 |   45    |
| 351214 |   40    |
| 102524 |   40    |


Символы ^ в спецификаторах формата {:^14} и {:^11} означают, что соответствующие значения должны быть выровнены по центру ячейки, ширина ячейки для столбца id составляет 14 символов, а для столбца minutes - 11 символов.

2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [22]:
from bs4 import BeautifulSoup
import pandas as pd

# Чтение XML файла
with open('steps_sample.xml', 'r') as f:
    reader = BeautifulSoup(f, 'xml')

# Чтение CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Создание словаря с шагами по каждому рецепту
ides = []
steps = []
for recipe in reader.recipes.find_all('recipe'):
    ides += [id.next for id in recipe.find_all('id')]
    steps.append([step.next for step in recipe.steps.find_all('step')])
dict_id_steps = {i: j for i, j in zip(ides, steps)}

# Функция для вывода информации о рецепте
def show_info(name, steps='', minutes='', author_id=''):
    str_ = f'"{name.title()}"\n\n'
    for i in range(1, len(steps)+1):
        str_ += f'{i}. {steps[i-1].capitalize()}\n'
    str_ += f'----------\n' + f'Автор: {author_id}\n' + f'Среднее время приготовления: {minutes} минут\n'
    return str_

# Пример вызова функции
show_info(
    name="george s at the cove black bean soup",
    steps=[
        "clean the leeks and discard the dark green portions",
        "cut the leeks lengthwise then into one-inch pieces",
        "melt the butter in a medium skillet , med",
    ],
    minutes=90,
    author_id=35193,
)


'"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'

In [24]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [25]:
# Получение шагов для рецепта с id=25082
steps = dict_id_steps['25082']

# Поиск строк, содержащих время приготовления в формате "X часов/минут"
for i in steps:
    time_match = re.findall(r'\d+ (?:hour[s]?|minute[s]?)', i )
    if time_match:
        print(time_match)


['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [26]:
# Фильтр строк
samples = recipes.dropna()[recipes.dropna().description.str.contains('^this[A-Za-z0-9.,?! ]+, ?but')]['description']

# Вывести количество найденных строк
print(len(samples))

# Вывести первые 3 строки из найденных
for i in range(3):
    print(samples.iloc[i])


155
this cookie is made without flour and without sweetened condensed milk.  light and chewy.  i prefer using unsweetened coconut, but sweetened is fine.  this was my grandmother's recipe... probably from better homes and gardens.  they are flat, not the traditional mounded cookie.  perfect for dipping!
this is a moist, buttery apple cake.  the dough is best made the night before, put in the fridge and then filled and baked the next morning.
this is a very refreshing, nice summer dessert. very simple to prepare. you can use any pie filling that you wish, but we like lemon the best...it makes a nice contrast with the sweetness of the cream cheese mixture.


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [29]:
# Получить список шагов по рецепту с id = 72367
steps = dict_id_steps['72367']

# Заменить все вхождения "цифра / цифра" на "цифра/цифра"
for i, step in enumerate(steps):
    steps[i] = re.sub(r'(\d) (\/) (\d)', r'\1\2\3', step)

# Вывести список шагов с измененными записями
for step in steps:
    print(step)


mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

In [30]:
import nltk

In [31]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [33]:
with open('steps_sample.xml', 'r') as f:
    reader = BeautifulSoup(f, 'xml')

# Создать список шагов для каждого рецепта
steps = []
for recipe in reader.recipes.find_all('recipe'):
    steps.append([step.next for step in recipe.steps.find_all('step')])

# Составить список всех слов в шагах
list_words = nltk.word_tokenize(' '.join([' '.join(i) for i in steps]))

# Оставить только уникальные слова, состоящие только из букв, и подсчитать их количество
unique_words = set(i.lower() for i in list_words if i.isalpha())
print(len(unique_words))

14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [36]:
recipes = pd.read_csv('recipes_sample.csv')

# Добавляем колонку с количеством предложений в описании рецепта
recipes['n_sent'] = recipes['description'].apply(lambda i: len(nltk.sent_tokenize(str(i))))

# Выводим пять рецептов с наибольшим количеством предложений в описании
top_recipes = recipes.nlargest(5, ['n_sent'])
top_recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_sent
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,76
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,27
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,24
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,23
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,23


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [39]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [40]:
# Получение имени рецепта по его ID
recipes[recipes['id'] == 241106]['name'].values

# Функция для разметки частей речи в предложении
def sent_tag(sent):
  word_value = nltk.pos_tag(nltk.word_tokenize(sent))
  up, down = '', ''
  for i in word_value:
    up += f' {i[1]: ^{len(i[0])}}'
    down += i[0]+' '
  return up+'\n'+down

# Пример использования функции
sent = 'eggplant steaks with chickpeas  feta cheese and black olives'
print(sent_tag(sent))


    JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
